In [1]:
from fastai.text.all import *
import os
import pandas as pd
%config Completer.use_jedi = False

In [2]:
# Read into a Data Frame
df_valid = pd.read_csv('data/valid_en_fr.csv')

# Add a is_valid colum. The ColSplitter() function below expects this column.
df_valid['is_valid'] = True

# View our work
df_valid.head()

,Unnamed: 0,src_english,trg_french,is_valid
0,0,A group of men are loading cotton onto a truck,Un groupe d'hommes chargent du coton dans un camion,True
1,1,A man sleeping in a green room on a couch.,Un homme dormant dans une chambre verte sur un canapé.,True
2,2,A boy wearing headphones sits on a woman's shoulders.,Un garçon avec un casque est assis sur les épaules d'une femme.,True
3,3,Two men setting up a blue ice fishing hut on an iced over lake,Deux hommes installant une tente de pêche sur glace bleue sur un lac gelé,True
4,4,A balding man wearing a red life jacket is sitting in a small boat.,Un homme chauve vêtu d'un gilet de sauvetage rouge est assis dans un petit bateau.,True


In [3]:
# Read into a Data Frame
df_train = pd.read_csv('data/train_en_fr.csv')

# Add a is_valid colum. The ColSplitter() function below expects this column.
df_train['is_valid'] = False

# View our work
df_train.head()

,Unnamed: 0,src_english,trg_french,is_valid
0,0,"Two young, White males are outside near many bushes.",Deux jeunes hommes blancs sont dehors près de buissons.,False
1,1,Several men in hard hats are operating a giant pulley system.,Plusieurs hommes en casque font fonctionner un système de poulies géant.,False
2,2,A little girl climbing into a wooden playhouse.,Une petite fille grimpe dans une maisonnette en bois.,False
3,3,A man in a blue shirt is standing on a ladder cleaning a window.,Un homme dans une chemise bleue se tient sur une échelle pour nettoyer une fenêtre.,False
4,4,Two men are at the stove preparing food.,Deux hommes aux fourneaux préparent à manger.,False


In [4]:
# Stack the two dataframes into one.
df = pd.concat([df_train,df_valid], ignore_index=True)
df.head()

,Unnamed: 0,src_english,trg_french,is_valid
0,0,"Two young, White males are outside near many bushes.",Deux jeunes hommes blancs sont dehors près de buissons.,False
1,1,Several men in hard hats are operating a giant pulley system.,Plusieurs hommes en casque font fonctionner un système de poulies géant.,False
2,2,A little girl climbing into a wooden playhouse.,Une petite fille grimpe dans une maisonnette en bois.,False
3,3,A man in a blue shirt is standing on a ladder cleaning a window.,Un homme dans une chemise bleue se tient sur une échelle pour nettoyer une fenêtre.,False
4,4,Two men are at the stove preparing food.,Deux hommes aux fourneaux préparent à manger.,False


In [5]:
defaults.text_proc_rules.copy()

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [6]:
custom_proc_rules = defaults.text_proc_rules.copy()
custom_proc_rules[-1] = partial(lowercase,add_bos=True,add_eos=True)
custom_proc_rules = [custom_proc_rules[-1]]

In [7]:
logs = DataBlock(
    
    # blocks specify what type of data we are going to be loading.
    # In this case both are text files contained in the same df
    blocks=(
        TextBlock.from_df('src_english', is_lm=False, rules=custom_proc_rules),
        TextBlock.from_df('trg_french', is_lm=False, rules=custom_proc_rules)),
    
    # The TestBlock tokenization process puts tokenized inputs into a column called text. 
    # The ColReader for get_x will always reference text, even if the original text inputs 
    # were in a column with another name in the dataframe.
    get_x=ColReader('text'),
    get_y=ColReader('text'),
    
    # The dataframe needs to have a is_valid column for this to work.
    splitter=ColSplitter()

)

In [8]:
dls = logs.dataloaders(df, bs=64, seq_len=100)


/home/kpawlikowski/anaconda3/envs/deeplearning_pytorch/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [9]:
dls.show_batch()

,text,text_
0,"xxbos two men on motorcycles , one with a silver helmet and the other with a black helmet , riding in a city , with a boy with a white long - sleeve shirt on , approaching one of the motorcycles . xxeos","xxbos deux hommes à moto , l'un avec un casque argenté et l'autre avec un casque noir , circulant dans une ville , avec un garçon en chemise à manches longues blanche approchant de l'une des motos . xxeos"
1,"xxbos a soccer player in a green uniform with a ball in is hands is being held up by some of his teammates , while an opposing player in red reaches for the ball . xxeos",xxbos un joueur de football en maillot vert avec un ballon dans les mains est porté par quelques uns de ses coéquipiers pendant qu'un joueur adverse en rouge tente d'attraper le ballon . xxeos
2,"xxbos a male in a black shirt , and black pants , working on the engine of an old , green antique automobile , with a yellow gas canister sitting on the grass . xxeos","xxbos un homme en t - shirt noir et pantalon noir , travaille sur le moteur d'une vieille automobile vert xxunk , avec un xxunk jaune dans l'herbe xxeos"
3,"xxbos two men , one being john xxunk , and one woman sitting in red rolling chairs next to a white sign with a blue letter d and a number 5 on it . xxeos","xxbos deux hommes , l'un étant john xxunk , et une femme assis dans des fauteuils à roulettes rouges à côté d'un panneau blanc avec une xxunk xxunk bleue et un numéro cinq dessus . xxeos"
4,"xxbos two boys are standing in the middle of the street , the one with black shorts is standing back clapping his hands together while the other on in gray is running forward . xxeos","xxbos deux garçons sont debout au milieu de la rue , l'un avec un short noir , debout en applaudissant tandis que l'autre , en gris avance en courant . xxeos"
5,"xxbos three men playing football where one guy wearing a red uniform is holding the football , the second guy wearing a red uniform and the third guy wearing a white uniform . xxeos","xxbos trois hommes jouant au football américain ; un gars avec un maillot rouge tient le ballon , le deuxième porte un maillot rouge et le troisième un maillot blanc . xxeos"
6,xxbos a person in a large pink triangle - shaped costume waves as a man and a woman dressed in matching brightly - colored striped and polka - dotted outfits smile back . xxeos,"xxbos une personne dans un grand costume rose en forme de triangle agite la main , tandis qu'un homme et une femme tous deux vêtus de vêtements à pois et à rayures de couleur brillante sourient . xxeos"
7,"xxbos many hockey players are xxunk up around the goal , the goalie is in a crouched position and one player wearing a red and white jersey appears to be celebrating . xxeos","xxbos de nombreux joueurs de hockey sont regroupés autour du but , le gardien est accroupi et un joueur avec un maillot rouge et blanc semble xxunk quelque chose . xxeos"
8,"xxbos people from infants to the elderly seem excited as one woman in a yellow shirt jumps off the ground in front of a sign saying "" xxunk welcome "" . xxeos","xxbos des gens de tous les âges semblent xxunk tandis qu'une femme en t - shirt jaune saute devant une banderole disant "" bienvenue xxunk "" . xxeos"


In [10]:
# View our training Dataset. Notice how the dataset is now Numericalized
original_vocab_len = len(dls.train.vocab[0])
original_vocab_len

4640

In [11]:
# View our validation Dataset. Notice how the dataset is now Numericalized
template_vocab_len = len(dls.train.vocab[1])
template_vocab_len

5088

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout_p):
        super(Encoder,self).__init__()
        '''
            input_size:     size of the input vocabulary
            embedding_size: size of each word embedding
            hidden_size:    size of the hidden layer
            num_layers:     number of layers in our encoder lstm
            dropout:        the probability for our dropout
        '''
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.dropout = nn.Dropout(dropout_p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers,dropout=dropout_p)
        
    def forward(self,x):

        embedding = self.dropout(self.embedding(x))
        
        outputs, (hidden, cell) = self.rnn(embedding)
   
        return hidden,cell


In [13]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout_p):
        super(Decoder,self).__init__()
        '''
            input_size:     size of the input vocabulary
            embedding_size: size of each word embedding
            hidden_size:    size of the hidden layer
            output_size:    size of the output vocabulary
            num_layers:     number of layers in our encoder lstm
            dropout:        the probability for our dropout
        '''
        
        self.output_size = output_size
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.dropout=nn.Dropout(dropout_p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers,dropout=dropout_p)
        self.fc = nn.Linear(hidden_size,output_size)
    
    def forward(self,x,hidden,cell):
        # shape of x: (N) but we want (1,N)
        x = x.unsqueeze(0)
        
        # embedding shape: (1, N, embedding_size)
        embedding = self.dropout(self.embedding(x))
        
        # shape of outputs: (1, N, hidden_size)
        outputs, (hidden,cell) = self.rnn(embedding,(hidden,cell))
        
        # shape of predictions: (1, N, length_of_vocab)
        predictions = self.fc(outputs)
        
        predictions = predictions.squeeze(0)
        
        return predictions, hidden, cell
        

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder,decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, source, target, teacher_force_ratio = 0.5):
        batch_size = source.shape[1]
        
        target_len = target.shape[0]
        
        target_vocab_size = self.decoder.output_size
  

        outputs = torch.zeros(target_len,batch_size,target_vocab_size).to(device)

        hidden, cell = self.encoder(source)
        
        # Grab the start token
        x = target[0]
        
        for t in range(1,target_len):
 
            output, hidden, cell = self.decoder(x,hidden,cell)

            outputs[t] = output
    
            best_guess = output.argmax(1)
            
            x = target[t] if random.random() < teacher_force_ratio else best_guess
        
        return outputs

In [15]:
class TeacherForcingCallback(Callback):
    def __init__(self, teacher_forcing_ratio=0):
        self.teacher_forcing_ratio = teacher_forcing_ratio
    """
    Callback that sends the y's to the model too
    """
    def before_batch(self):
        # print('before_batch 2')
        x,y = self.x, self.y
        self.learn.xb = (x,y,self.teacher_forcing_ratio)
        

In [16]:
class LearningRatePrinter(Callback):
    
    def before_train(self):
        lr = self.opt.hypers
        print(f' before_train learning_rate: {lr}')


In [17]:
class DebuggerCallBack(Callback):
    def on_train_begin(self):
        import pdb
        pdb.set_trace()

In [18]:
def seq2seq_acc(out, targ, pad_idx=1):
    out = out.argmax(2)
    targ_len = targ.size()
    out_len= out.size()
    if targ_len>out_len: out  = F.pad(out,  (0,0,0,targ_len-out_len,0,0), value=pad_idx)
    if out_len>targ_len: targ = F.pad(targ, (0,out_len-targ_len,0,0), value=pad_idx)
    return (out==targ).float().mean()

In [19]:
# Training hyperparameters
num_epoc = 10
learning_rate = 0.0001
batch_size = 64

In [20]:
# Model hyperparameters

INPUT_DIM = len(dls.train.vocab[0])
OUTPUT_DIM = len(dls.train.vocab[1])
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
HID_DIM = 1024
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

In [21]:
enc = Encoder(INPUT_DIM,  ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, OUTPUT_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec).to(device)

In [22]:
criterion = CrossEntropyLossFlat(ignore_index=1)

In [23]:

learn = Learner(dls, model, loss_func=criterion, metrics=[seq2seq_acc]).to_fp16()
#learn = Learner(dls, model, loss_func=criterion, metrics=[seq2seq_acc],cbs=TeacherForcingCallback(teacher_forcing_ratio=0.4)).to_fp16()

In [24]:
learn.fit(
    lr=0.0001,
    n_epoch=10,
    cbs=[
         TeacherForcingCallback(teacher_forcing_ratio=0),
         SaveModelCallback(monitor='seq2seq_acc',fname='seq2seqkristest2'),
        LearningRatePrinter()
    ]
)

epoch,train_loss,valid_loss,seq2seq_acc,time
0,4.834180,4.842516,0.147491,01:52
1,4.715242,4.664337,0.157062,01:51
2,4.715970,4.644075,0.161072,01:51
3,4.644243,4.605755,0.160774,01:51
4,4.601316,4.593852,0.160000,01:54
5,4.599597,4.589972,0.162078,01:53
6,4.618322,4.582973,0.160703,01:53
7,4.575459,4.577770,0.161178,01:54
8,4.594923,4.578576,0.161595,01:54
9,4.656511,4.581444,0.160858,01:58


 before_train learning_rate: [{'wd': 0.01, 'sqr_mom': 0.99, 'lr': 0.0001, 'mom': 0.9, 'eps': 1e-05}]
Better model found at epoch 0 with seq2seq_acc value: 0.1474912464618683.
 before_train learning_rate: [{'wd': 0.01, 'sqr_mom': 0.99, 'lr': 0.0001, 'mom': 0.9, 'eps': 1e-05}]
Better model found at epoch 1 with seq2seq_acc value: 0.15706218779087067.
 before_train learning_rate: [{'wd': 0.01, 'sqr_mom': 0.99, 'lr': 0.0001, 'mom': 0.9, 'eps': 1e-05}]
Better model found at epoch 2 with seq2seq_acc value: 0.16107219457626343.
 before_train learning_rate: [{'wd': 0.01, 'sqr_mom': 0.99, 'lr': 0.0001, 'mom': 0.9, 'eps': 1e-05}]
 before_train learning_rate: [{'wd': 0.01, 'sqr_mom': 0.99, 'lr': 0.0001, 'mom': 0.9, 'eps': 1e-05}]
 before_train learning_rate: [{'wd': 0.01, 'sqr_mom': 0.99, 'lr': 0.0001, 'mom': 0.9, 'eps': 1e-05}]
Better model found at epoch 5 with seq2seq_acc value: 0.16207760572433472.
 before_train learning_rate: [{'wd': 0.01, 'sqr_mom': 0.99, 'lr': 0.0001, 'mom': 0.9, 'eps': 1e

In [25]:
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(4640, 300)
    (rnn): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5088, 300)
    (rnn): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=5088, bias=True)
  )
)

In [26]:
translate_me = 'A group of men are loading cotton onto a truck'

In [27]:
dl = learn.dls.test_dl([translate_me],batch_size=1)

In [28]:
dl.show_batch()

,text
0,xxbos a group of men are loading cotton onto a truck xxeos


In [29]:
sentence_tensor = torch.LongTensor(dl.dataset[0][0]).unsqueeze(1).to(device)

In [30]:
sentence_tensor.shape

torch.Size([12, 1])

In [32]:
with torch.no_grad():
    hidden, cell = model.encoder(sentence_tensor)

max_length=100
# Add xxbos
outputs = [2]

for _ in range(max_length):
    previous_word = torch.LongTensor([outputs[-1]]).to(device)

    with torch.no_grad():
        output, hidden, cell = model.decoder(previous_word, hidden, cell)
        best_guess = output.argmax(1).item()
        print(best_guess)
        outputs.append(best_guess)
    print(outputs)
    # Model predicts it's the end of the sentence
    if output.argmax(1).item() == 3:
        break

13
[2, 13]
13
[2, 13, 13]
13
[2, 13, 13, 13]
13
[2, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
13
[2, 13, 13, 13, 1

In [33]:
learn.dls.decode(outputs)

(TensorText(2),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(13),
 TensorText(12),
 TensorText(13),
 TensorText(13),
 TensorText(12),
 TensorText(13),
 TensorText(13),
 TensorText(12),
 TensorText(13),
 TensorText(13),
 TensorText(12),
 TensorText(13),
 TensorText(13),
 TensorText(12)

# learn.dls.vocab